# Perkeliamas mokymas 

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
import sys
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [ ]:
from helper_evaluation import set_all_seeds, set_deterministic, compute_confusion_matrix
from helper_train_extended import train_model
from helper_plotting_extended import plot_training_loss, plot_accuracy, show_examples, plot_confusion_matrix
from helper_dataset_extended import get_dataloaders_cifar10, UnNormalize

In [ ]:
RANDOM_SEED = 123
BATCH_SIZE = 256
NUM_EPOCHS = 50
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# https://pytorch.org/vision/stable/models/generated/torchvision.models.vgg16.html#torchvision.models.vgg16
# mean=[0.485, 0.456, 0.406] and std=[0.229, 0.224, 0.225].

train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),                 
    torchvision.transforms.ToTensor(),                
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.2255))])


train_loader, valid_loader, test_loader = get_dataloaders_cifar10(
    batch_size=BATCH_SIZE,
    validation_fraction=0.1,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    num_workers=2)


for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break

Kaip sužinoti savo duomenų rinkinio įrašų reikšmių vidurkį ir standartinį nuokrypį:

In [ ]:
train_mean = []
train_std  = []
for i, image in enumerate(train_loader, 0):
    numpy_image = image[0].numpy()
    batch_mean  = np.mean(numpy_image, axis=(0, 2, 3))
    batch_std   = np.std(numpy_image, axis=(0, 2, 3))
    train_mean.append(batch_mean)
    train_std.append(batch_std)
    
train_mean = np.mean(train_mean, axis=0)
train_std  = np.std(train_std, axis=0)

print(f'mean: {train_mean}')
print(f'std : {train_std}')

In [ ]:
model = torchvision.models.vgg16(pretrained=True)
model

## Modelio svorių užšaldymas

Mes ir toliau galėsime naudoti `forward` ir `backward` metodus bet parametrai nebus atnaujinami.

In [ ]:
for param in model.parameters():
    param.requires_grad = False

Tarkime, kad norime sureguliuoti (išmokyti) tik paskutinius 3 sluoksnius:

In [ ]:
model.classifier[0].requires_grad = True
model.classifier[3].requires_grad = True

Paskutiniame sluoksnyje, kadangi klasių etikečių skaičius skiriasi nuo ImageNet, išvesties sluoksnį pakeičiame taip kad atitiktu jūsų duomenų rinkio išvestis:

In [ ]:
model.classifier[6] = torch.nn.Linear(4096, 10)

## Treniravimas

In [ ]:
model = model.to(DEVICE)

optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       verbose=True)

minibatch_loss_list, train_acc_list, valid_acc_list = train_model(
    model=model,
    num_epochs=NUM_EPOCHS,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    device=DEVICE,
    scheduler=scheduler,
    scheduler_on='valid_acc',
    logging_interval=100)

plot_training_loss(minibatch_loss_list=minibatch_loss_list,
                   num_epochs=NUM_EPOCHS,
                   iter_per_epoch=len(train_loader),
                   results_dir=None,
                   averaging_iterations=200)
plt.show()

plot_accuracy(train_acc_list=train_acc_list,
              valid_acc_list=valid_acc_list,
              results_dir=None)
plt.ylim([60, 100])
plt.show()

In [ ]:
model.cpu()
unnormalizer = UnNormalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.2255))
class_dict = {0: 'airplane',
              1: 'automobile',
              2: 'bird',
              3: 'cat',
              4: 'deer',
              5: 'dog',
              6: 'frog',
              7: 'horse',
              8: 'ship',
              9: 'truck'}

show_examples(model=model, data_loader=test_loader, unnormalizer=unnormalizer, class_dict=class_dict)

In [ ]:
mat = compute_confusion_matrix(model=model, data_loader=test_loader, device=torch.device('cpu'))
plot_confusion_matrix(mat, class_names=class_dict.values())
plt.show()